In [3]:
# Install required packages
%pip install pandas numpy matplotlib seaborn scikit-learn jupyter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
"""
🎛️ FEATURE ENGINEERING FOR PREDICTIVE MAINTENANCE
Purpose: Create new features from raw data to improve model performance
Based on EDA findings, we engineer features that capture equipment degradation patterns
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("=" * 60)
print("FEATURE ENGINEERING PIPELINE")
print("=" * 60)

# Load the data
df = pd.read_csv(r'C:\Users\pavan\Documents\Github P1\predictive-maintenace-system\machine predictive maintence classification\predictive_maintenance.csv')
print(f"📊 Original dataset shape: {df.shape}")
print(f"📋 Columns: {', '.join(df.columns.tolist()[:5])}...")  # Show first 5 columns

FEATURE ENGINEERING PIPELINE
📊 Original dataset shape: (10000, 10)
📋 Columns: UDI, Product ID, Type, Air temperature [K], Process temperature [K]...


In [5]:
"""
📉 DEGRADATION INDICATORS
Purpose: Create features that capture equipment wear and tear over time
These features help models detect gradual failure patterns
"""

# Make a copy to preserve original data
df_features = df.copy()

print("\n1️⃣ CREATING DEGRADATION INDICATORS...")

# 1. Tool Wear Rate (normalized tool wear)
df_features['tool_wear_rate'] = df_features['Tool wear [min]'] / df_features['Tool wear [min]'].max()

# 2. Temperature Difference (important for thermal failures)
df_features['temp_difference'] = df_features['Process temperature [K]'] - df_features['Air temperature [K]']

# 3. Power Metric (Rotational speed × Torque)
df_features['power_estimate'] = (df_features['Rotational speed [rpm]'] * df_features['Torque [Nm]']) / 1000

# 4. Machine Load (combined metric)
df_features['machine_load'] = (df_features['Tool wear [min]'] / 100) * (df_features['power_estimate'] / 100)

print("✅ Created 4 degradation features")
print("   - tool_wear_rate: Normalized tool wear (0 to 1)")
print("   - temp_difference: Process vs Air temperature delta")
print("   - power_estimate: Rotational speed × Torque")
print("   - machine_load: Combined wear and power metric")


1️⃣ CREATING DEGRADATION INDICATORS...
✅ Created 4 degradation features
   - tool_wear_rate: Normalized tool wear (0 to 1)
   - temp_difference: Process vs Air temperature delta
   - power_estimate: Rotational speed × Torque
   - machine_load: Combined wear and power metric


In [6]:
"""
🔄 INTERACTION FEATURES
Purpose: Capture relationships between sensors that signal problems
Based on EDA correlation findings (-0.88 between speed and torque)
"""

print("\n2️⃣ CREATING INTERACTION FEATURES...")

# 1. Speed-Torque Ratio (captures their strong negative correlation from EDA)
df_features['speed_torque_ratio'] = df_features['Rotational speed [rpm]'] / (df_features['Torque [Nm]'] + 0.1)

# 2. Temperature Average
df_features['avg_temperature'] = (df_features['Air temperature [K]'] + df_features['Process temperature [K]']) / 2

# 3. Thermal Stress (temperature difference × power)
df_features['thermal_stress'] = df_features['temp_difference'] * df_features['power_estimate']

# 4. Normalized Torque (relative to max)
df_features['torque_normalized'] = df_features['Torque [Nm]'] / df_features['Torque [Nm]'].max()

print("✅ Created 4 interaction features")
print("   - speed_torque_ratio: Captures -0.88 correlation from EDA")
print("   - avg_temperature: Average thermal condition")
print("   - thermal_stress: Combined thermal and power stress")
print("   - torque_normalized: Scaled torque (0 to 1)")


2️⃣ CREATING INTERACTION FEATURES...
✅ Created 4 interaction features
   - speed_torque_ratio: Captures -0.88 correlation from EDA
   - avg_temperature: Average thermal condition
   - thermal_stress: Combined thermal and power stress
   - torque_normalized: Scaled torque (0 to 1)


In [7]:
"""
📊 STATISTICAL FEATURES
Purpose: Create rolling statistics that capture trends over time
These help detect gradual changes that lead to failures
"""

print("\n3️⃣ CREATING STATISTICAL FEATURES...")

# Since we don't have time series, we'll create simulated "operating cycles"
# Create a dummy time index based on UDI
df_features = df_features.sort_values('UDI').reset_index(drop=True)

# Create rolling window features (simulating recent history)
window_size = 50

# Rolling averages for key sensors
df_features['rolling_avg_torque'] = df_features['Torque [Nm]'].rolling(window=window_size, min_periods=1).mean()
df_features['rolling_avg_speed'] = df_features['Rotational speed [rpm]'].rolling(window=window_size, min_periods=1).mean()
df_features['rolling_avg_temp'] = df_features['avg_temperature'].rolling(window=window_size, min_periods=1).mean()

# Rolling standard deviations (capture variability)
df_features['rolling_std_torque'] = df_features['Torque [Nm]'].rolling(window=window_size, min_periods=1).std()
df_features['rolling_std_speed'] = df_features['Rotational speed [rpm]'].rolling(window=window_size, min_periods=1).std()

# Trend features (slope of recent values)
for i in range(1, 6):
    df_features[f'torque_trend_{i}'] = df_features['Torque [Nm]'].diff(i)

print("✅ Created 8 statistical/trend features")
print("   - Rolling averages (torque, speed, temp)")
print("   - Rolling standard deviations")
print("   - Torque trends (1-5 step differences)")


3️⃣ CREATING STATISTICAL FEATURES...
✅ Created 8 statistical/trend features
   - Rolling averages (torque, speed, temp)
   - Rolling standard deviations
   - Torque trends (1-5 step differences)


In [8]:
"""
🎯 FAILURE-SPECIFIC FEATURES
Purpose: Create features targeting specific failure types from EDA
Based on failure type analysis: Heat Dissipation, Power, Overstrain, Tool Wear
"""

print("\n4️⃣ CREATING FAILURE-SPECIFIC FEATURES...")

# 1. Heat-related features (for Heat Dissipation failures - most common)
df_features['heat_risk_score'] = (df_features['temp_difference'] * df_features['avg_temperature']) / 1000

# 2. Power stress features (for Power failures)
df_features['power_variability'] = df_features['rolling_std_speed'] * df_features['rolling_std_torque']

# 3. Mechanical stress (for Overstrain failures)
df_features['mechanical_stress'] = df_features['Torque [Nm]'] * df_features['Rotational speed [rpm]'] / 10000

# 4. Wear progression (for Tool Wear failures)
df_features['wear_progression'] = df_features['Tool wear [min]'] * df_features['tool_wear_rate']

# 5. Combined risk score
df_features['combined_risk'] = (
    df_features['heat_risk_score'] + 
    df_features['power_variability'] + 
    df_features['mechanical_stress'] + 
    df_features['wear_progression']
) / 4

print("✅ Created 5 failure-specific features")
print("   - heat_risk_score: For Heat Dissipation failures (112 cases)")
print("   - power_variability: For Power failures (95 cases)")
print("   - mechanical_stress: For Overstrain failures (78 cases)")
print("   - wear_progression: For Tool Wear failures (45 cases)")
print("   - combined_risk: Overall failure risk score")


4️⃣ CREATING FAILURE-SPECIFIC FEATURES...
✅ Created 5 failure-specific features
   - heat_risk_score: For Heat Dissipation failures (112 cases)
   - power_variability: For Power failures (95 cases)
   - mechanical_stress: For Overstrain failures (78 cases)
   - wear_progression: For Tool Wear failures (45 cases)
   - combined_risk: Overall failure risk score


In [9]:
"""
🔢 ENCODING CATEGORICAL FEATURES
Purpose: Convert categorical data to numerical format for ML models
"""

print("\n5️⃣ ENCODING CATEGORICAL FEATURES...")

# Check for categorical columns
categorical_cols = df_features.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns found: {categorical_cols}")

# Encode Type column if it exists
if 'Type' in df_features.columns:
    type_mapping = {'L': 0, 'M': 1, 'H': 2}
    df_features['Type_encoded'] = df_features['Type'].map(type_mapping)
    print(f"   - Encoded 'Type' column: {type_mapping}")

# Encode Failure Type if it exists
if 'Failure Type' in df_features.columns:
    # Create binary flags for each failure type
    failure_types = df_features['Failure Type'].unique()
    for ftype in failure_types:
        if ftype != 'No Failure':
            col_name = f'failure_{ftype.lower().replace(" ", "_")}'
            df_features[col_name] = (df_features['Failure Type'] == ftype).astype(int)
    print(f"   - Created binary flags for {len(failure_types)-1} failure types")

print("✅ Encoding complete")


5️⃣ ENCODING CATEGORICAL FEATURES...
Categorical columns found: ['Product ID', 'Type', 'Failure Type']
   - Encoded 'Type' column: {'L': 0, 'M': 1, 'H': 2}
   - Created binary flags for 5 failure types
✅ Encoding complete


In [10]:
"""
📈 FEATURE ANALYSIS & VISUALIZATION
Purpose: Analyze the new features and their relationship with failures
"""

print("\n6️⃣ ANALYZING ENGINEERED FEATURES...")

# Check feature count
original_features = len(df.columns)
new_features = len(df_features.columns)
engineered_features = new_features - original_features

print(f"📊 Feature Count Summary:")
print(f"   - Original features: {original_features}")
print(f"   - Engineered features: {engineered_features}")
print(f"   - Total features: {new_features}")

# Show new features
new_feature_list = [col for col in df_features.columns if col not in df.columns]
print(f"\n🎯 New Features ({engineered_features} total):")
for i, feature in enumerate(new_feature_list, 1):
    print(f"   {i:2d}. {feature}")

# Correlation of new features with target
if 'Target' in df_features.columns:
    print("\n📈 Top 10 New Features Correlated with Target:")
    new_features_corr = df_features[new_feature_list + ['Target']].corr()['Target'].drop('Target')
    new_features_corr = new_features_corr.sort_values(key=abs, ascending=False)
    
    for i, (feature, corr) in enumerate(new_features_corr.head(10).items(), 1):
        print(f"   {i:2d}. {feature:25s}: {corr:+.3f}")


6️⃣ ANALYZING ENGINEERED FEATURES...
📊 Feature Count Summary:
   - Original features: 10
   - Engineered features: 29
   - Total features: 39

🎯 New Features (29 total):
    1. tool_wear_rate
    2. temp_difference
    3. power_estimate
    4. machine_load
    5. speed_torque_ratio
    6. avg_temperature
    7. thermal_stress
    8. torque_normalized
    9. rolling_avg_torque
   10. rolling_avg_speed
   11. rolling_avg_temp
   12. rolling_std_torque
   13. rolling_std_speed
   14. torque_trend_1
   15. torque_trend_2
   16. torque_trend_3
   17. torque_trend_4
   18. torque_trend_5
   19. heat_risk_score
   20. power_variability
   21. mechanical_stress
   22. wear_progression
   23. combined_risk
   24. Type_encoded
   25. failure_power_failure
   26. failure_tool_wear_failure
   27. failure_overstrain_failure
   28. failure_random_failures
   29. failure_heat_dissipation_failure

📈 Top 10 New Features Correlated with Target:
    1. failure_heat_dissipation_failure: +0.568
    2. fai

In [11]:
"""
📊 VISUALIZE NEW FEATURES
Purpose: Visualize how engineered features separate failures from non-failures
"""

print("\n7️⃣ VISUALIZING KEY ENGINEERED FEATURES...")

# Select top 4 new features by correlation
if 'Target' in df_features.columns and len(new_feature_list) >= 4:
    top_features = new_features_corr.head(4).index.tolist()
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for idx, feature in enumerate(top_features):
        # Create box plot
        sns.boxplot(data=df_features, x='Target', y=feature, ax=axes[idx], palette=['#2ecc71', '#e74c3c'])
        axes[idx].set_title(f'{feature} vs Failure', fontweight='bold')
        axes[idx].set_xlabel('Failure (0=No, 1=Yes)')
        axes[idx].set_ylabel(feature)
        
        # Add mean lines
        for target_val in [0, 1]:
            mean_val = df_features[df_features['Target'] == target_val][feature].mean()
            axes[idx].axhline(mean_val, color='black' if target_val == 0 else 'yellow', 
                            linestyle='--', alpha=0.7, linewidth=1)
    
    plt.suptitle('Engineered Features: Failure vs Non-Failure Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Not enough features or target column not found for visualization")


7️⃣ VISUALIZING KEY ENGINEERED FEATURES...


In [13]:
"""
💾 SAVE ENGINEERED DATASET
Purpose: Save the enhanced dataset for modeling phase
"""

print("\n8️⃣ SAVING ENGINEERED DATASET...")

import os

# ✅ MODIFIED: Use your exact path
base_path = r'C:\Users\pavan\Documents\Github P1\predictive-maintenace-system'

# Create processed data directory in YOUR path
processed_dir = os.path.join(base_path, 'data', 'processed')
os.makedirs(processed_dir, exist_ok=True)

# Save the engineered dataset to YOUR path
output_path = os.path.join(processed_dir, 'predictive_maintenance_engineered.csv')
df_features.to_csv(output_path, index=False)

print(f"✅ Dataset saved to: {output_path}")
print(f"   - Original shape: {df.shape}")
print(f"   - Engineered shape: {df_features.shape}")
print(f"   - New features added: {engineered_features}")

# Verify file was created
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / 1024  # Size in KB
    print(f"   - File size: {file_size:.2f} KB")
else:
    print("   ⚠️  Warning: File may not have been created")

# Create a feature summary
summary_df = pd.DataFrame({
    'Feature_Type': ['Original', 'Engineered', 'Total'],
    'Count': [original_features, engineered_features, new_features]
})

print("\n📊 FINAL SUMMARY:")
display(summary_df)

print("\n🎉 FEATURE ENGINEERING COMPLETE!")
print("Next step: Modeling with enhanced dataset")


8️⃣ SAVING ENGINEERED DATASET...
✅ Dataset saved to: C:\Users\pavan\Documents\Github P1\predictive-maintenace-system\data\processed\predictive_maintenance_engineered.csv
   - Original shape: (10000, 10)
   - Engineered shape: (10000, 39)
   - New features added: 29
   - File size: 4019.89 KB

📊 FINAL SUMMARY:


Feature_Type  Count
0     Original     10
1   Engineered     29
2        Total     39


🎉 FEATURE ENGINEERING COMPLETE!
Next step: Modeling with enhanced dataset
